In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import set_random_seed

In [2]:
np.random.seed(1234567)
set_random_seed(678901)

In [3]:
df = pd.read_csv('diabetes.csv')

In [5]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy='mean', missing_values=0)

impute_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[impute_features] = si.fit_transform(df[impute_features])

dataset = df.to_numpy()

dataset_x = dataset[:, :-1]
dataset_y = dataset[:, -1]

In [6]:
from sklearn.model_selection import train_test_split

training_dataset_x, test_dataset_x, training_dataset_y, test_dataset_y = train_test_split(dataset_x, dataset_y, test_size=0.2)

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(name='Diabetes')

model.add(Input((training_dataset_x.shape[1],)))
model.add(Dense(16, activation='relu', name='Hidden-1'))
model.add(Dense(16, activation='relu', name='Hidden-2'))
model.add(Dense(1, activation='sigmoid', name='Output'))
model.summary()

Model: "Diabetes"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Hidden-1 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden-2 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 433 (1.69 KB)

 Trainable params: 433 (1.69 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.fit(training_dataset_x, training_dataset_y, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - binary_accuracy: 0.3653 - loss: 15.6973 - val_binary_accuracy: 0.3577 - val_loss: 6.8783
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.4658 - loss: 5.0253 - val_binary_accuracy: 0.6179 - val_loss: 1.1255
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.6178 - loss: 1.0298 - val_binary_accuracy: 0.6098 - val_loss: 0.9034
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.6617 - loss: 0.7999 - val_binary_accuracy: 0.6179 - val_loss: 0.8977
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.6722 - loss: 0.7545 - val_binary_accuracy: 0.6016 - val_loss: 0.8787
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.6796 - loss: 0.7242 - val_binary_accuracy: 0.6016 - val_loss: 0.8617
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - binary_accuracy: 0.6749 - loss: 0.7013 - val_binary_accuracy: 0.5935 - val_loss: 0.8417
Epoch 8/100

In [10]:
eval_result = model.evaluate(test_dataset_x, test_dataset_y, batch_size=32)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6817 - loss: 0.8036 


In [11]:
for i in range(len(eval_result)):
    print(f'{model.metrics_names[i]}: {eval_result[i]}')

loss: 0.8303808569908142
compile_metrics: 0.7077922224998474


In [12]:
predict_dataset = np.array([[2 ,90, 68, 12, 120, 38.2, 0.503, 28],
                            [4, 111, 79, 47, 207, 37.1, 1.39, 56],
                            [3, 190, 65, 25, 130, 34, 0.271, 26],
                            [8, 176, 90, 34, 300, 50.7, 0.467, 58],
                            [7, 106, 92, 18, 200, 35, 0.300, 48]])

predict_result = model.predict(predict_dataset)
print(predict_result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[0.5016371 ]
 [0.37216964]
 [0.8520156 ]
 [0.35476014]
 [0.12474416]]


In [13]:
for result in predict_result[:, 0]:
    print('Şeker hastasi' if result > 0.5 else 'Şeker Hastasi Değil')

Şeker hastasi
Şeker Hastasi Değil
Şeker hastasi
Şeker Hastasi Değil
Şeker Hastasi Değil
